# uploader to widget image

In [1]:
# uploader to image widget
import ipywidgets as widgets
from IPython.display import display
import time
import threading
import warnings

In [2]:
# uplaoder (in this tutorial, we only cover single image uploader)
uploader = widgets.FileUpload(
    accept='image/*',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
    )

In [3]:
# uploader image container
uploader_image_container = widgets.Box(
    children = ()
)

In [4]:
# wrapper
app = widgets.VBox(
    children = (uploader, uploader_image_container)
)

In [5]:
# image widget from uploader
def image_widget_from_uploader(uploader):
    if uploader.value:
        file_content, file_extension = get_content_and_extension(uploader)
        return widgets.Image(value=file_content, format=file_extension)
    else:
        warnings.warn("NO IMAGE HAS BEEN UPLOADED", category=Warning)
        return widgets.Image()

In [6]:
# uploader image content and extension
def get_content_and_extension(uploader):
    for key, value in uploader.value.items():
        return value['content'], value['metadata']['name'].split('.')[-1]

In [7]:
# main loop
def work():
    # global flag
    global thread_status
    
    # save uploader.value for state reference
    current_uploader_content = uploader.value
    
    while thread_status:
        
        # check if there is a change in uploader.value
        if current_uploader_content != uploader.value:
            
            # renew state reference
            current_uploader_content = uploader.value

            # read image
            image_widget = image_widget_from_uploader(uploader)
            
            # renew container value
            uploader_image_container.children = (image_widget,)
        
        # sleep for threading
        time.sleep(0.05)
        
        # exit flag from threading
        if thread_status == False:
            break

In [8]:
# reset flag
thread_status = False

# start flag
thread_status = True

# create thread
thread = threading.Thread(target=work)

# start thread
thread.start()

In [9]:
# display wrapper
display(app)

# extension: uploader to image in numpy

In [10]:
# uploader to image cv2
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [11]:
# image numpy from uploader
def image_numpy_from_uploader(uploader):
    file_content, extension = get_content_and_extension(uploader)
    arr_from_buf = np.frombuffer(file_content, np.uint8)
    return cv2.imdecode(arr_from_buf, cv2.IMREAD_COLOR), extension

In [12]:
# # to read uploader using cv2
# image_numpy, extension = image_numpy_from_uploader(uploader)
# plt.imshow(image_numpy)
# plt.show()

# extension: image in numpy to widget image

In [13]:
# image widget from image numpy
def image_widget_from_numpy(image_numpy, extension):
    _, encoded_image = cv2.imencode('.'+extension, image_numpy)
    image_bytes = encoded_image.tobytes()
    return widgets.Image(value=image_bytes, format=extension)

In [14]:
# # to convert image numpy to widget image 
# image_widget = image_widget_from_numpy(image_numpy, extension)
# display(image_widget)